In [3]:
from basic_rl_envi_for_dam_bidding_main.import_data_example import *

# Get Data
Bedeutung der einzelnen Variablen
- Forecasted Load: Day-ahead Total Load Forecast
- Solar: Day-ahead Generation Forecasts for Solar
- Wind Offshore: Day-ahead Generation Forecasts for Wind Offshore
- Wind Onshore: Day-ahead Generation Forecasts for Wind Onshore
- day ahead market price in EUR/MWh

In [6]:
start = pd.Timestamp(year=2023, month=3, day=1, tz="europe/brussels")
end = pd.Timestamp.now(tz="europe/brussels")
# get the end of the current day
end = end.floor('D') + pd.Timedelta(days=1)

In [8]:
df_demand = get_demand(start, end)
df_mcp = get_mcp(start, end)
df_vre = get_vre(start, end)
df_gen = get_gen(start, end)
#states = get_states_list(start, end)

In [18]:
# check last entry -> vre und gen nicht in real time verfügbar für unsere Forecasts
for d in [df_demand, df_mcp, df_vre, df_gen]:
    print(d.index[-1])

2023-05-11 23:45:00+02:00
2023-05-11 23:00:00+02:00
2023-05-10 23:45:00+02:00
2023-05-10 23:00:00+02:00


In [9]:
# get day ahead market price for each day
df_mcp_hourly = df_mcp.pivot_table(index=df_mcp.index.date, columns=df_mcp.index.hour, values=0)
df_mcp_hourly.index = pd.to_datetime(df_mcp_hourly.index)
df_mcp_hourly = df_mcp_hourly.add_prefix('market_price_hour_')
df_mcp_hourly['end_auction'] = (df_mcp_hourly.index - pd.Timedelta(hours = 12))
#df_demand: 2023-05-08 23:45:00+02:00 gibt den Load für 23:45-00:00 Uhr an

In [11]:
# get day ahead forecasted load for each hour of each day.
df_demand_hourly = df_demand.pivot_table(index=df_demand.index.date, columns=df_demand.index.hour, values='Forecasted Load', aggfunc='sum')
df_demand_hourly = df_demand_hourly.add_prefix('forecasted_load_hour_')

In [12]:
# get day ahead forecast for wind and solar
df_vre_agg = df_vre.groupby([pd.Grouper(freq='1h')]).sum()
df_vre_hourly = df_vre_agg.pivot_table(index=df_vre_agg.index.date, columns=df_vre_agg.index.hour, values=['Wind Onshore', 'Wind Offshore', 'Solar'], aggfunc='sum')
# add prefix and concatenate df
df_vre_hourly = pd.concat([df_vre_hourly[col].add_prefix(f'{col}_hour_') for col in df_vre_hourly.columns.get_level_values(level=0).unique()], axis=1)

In [14]:
# get Day-ahead Aggregated Generation
df_gen_hourly = df_gen.pivot_table(index=df_gen.index.date, columns=df_gen.index.hour, values='Actual Aggregated', aggfunc='sum')
df_gen_hourly = df_gen_hourly.add_prefix('actual_aggregated_hour_')

In [24]:
# join nur von df_demand und df_mcp -> die anderen Daten sind resultieren in Data Leakage
df = pd.concat([df_demand_hourly, df_mcp_hourly], axis=1)
df = df.reset_index().set_index('end_auction')
df = df.rename(columns={'index':'date'})

#df = pd.concat([df_demand_hourly, df_vre_hourly, df_gen_hourly, #df_mcp_hourly], axis=1)

In [25]:
df

,date,forecasted_load_hour_0,forecasted_load_hour_1,forecasted_load_hour_2,forecasted_load_hour_3,forecasted_load_hour_4,forecasted_load_hour_5,forecasted_load_hour_6,forecasted_load_hour_7,forecasted_load_hour_8,...,market_price_hour_14,market_price_hour_15,market_price_hour_16,market_price_hour_17,market_price_hour_18,market_price_hour_19,market_price_hour_20,market_price_hour_21,market_price_hour_22,market_price_hour_23
end_auction,,,,,,,,,,,,,,,,,,,,,
2023-02-28 12:00:00,2023-03-01,208696.0,202201.0,199722.0,201209.0,205615.0,217546.0,242339.0,259557.0,269253.0,...,116.09,122.81,135.46,162.74,183.69,182.48,162.74,144.85,142.05,131.51
2023-03-01 12:00:00,2023-03-02,210615.0,203958.0,201056.0,202722.0,207651.0,220122.0,245990.0,263352.0,273116.0,...,117.69,128.62,152.64,166.38,184.91,184.13,163.43,159.31,149.53,139.90
2023-03-02 12:00:00,2023-03-03,209231.0,202353.0,200215.0,201846.0,207412.0,219564.0,242874.0,261214.0,271378.0,...,110.89,114.69,129.31,155.98,165.00,152.94,133.31,122.20,112.09,101.63
2023-03-03 12:00:00,2023-03-04,208905.0,201157.0,197389.0,196285.0,196250.0,196526.0,201076.0,209615.0,221057.0,...,95.35,101.77,106.90,140.00,151.66,145.00,138.68,117.71,121.02,109.41
2023-03-04 12:00:00,2023-03-05,184430.0,178168.0,174895.0,174166.0,174403.0,175287.0,175933.0,182501.0,194247.0,...,110.22,115.14,129.37,145.14,166.67,172.37,160.23,152.49,143.66,130.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-06 12:00:00,2023-05-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.03,15.44,56.62,75.97,94.94,99.36,106.45,105.02,99.30,96.07
2023-05-07 12:00:00,2023-05-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.09,78.00,84.01,97.05,110.04,132.90,128.41,121.99,112.56,99.38
2023-05-08 12:00:00,2023-05-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68.29,80.80,84.44,100.06,116.34,121.93,121.05,110.97,104.67,89.93


In [27]:
X = df.loc[:,'forecasted_load_hour_0':'forecasted_load_hour_23']

In [31]:
y = df.loc[:,'market_price_hour_0':]